In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import regex as re

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA

In [88]:
movies = pd.read_csv('data/cleaned_movie_stats.csv')

movies.fillna(0, inplace = True)

In [109]:
conditions = [
    (movies['vote_average'] <= 4),
    (movies['vote_average'] > 4) & (movies['vote_average'] <= 7),
    (movies['vote_average'] > 7) & (movies['vote_average'] < 8.5),
    (movies['vote_average'] >= 8.5)
    ]

# create a list of the values we want to assign for each condition
values = ['bad', 'average', 'good', 'great']

# create a new column and use np.select to assign values to it using our lists as arguments
movies['vote_category'] = np.select(conditions, values)

In [110]:
movies.head()

,movie,vid_ids,views,likes,dislikes,comments,budget,genres,homepage,id,...,keyword,language,companies,countries,year,month,decade,homepage_binary,budget_mln,vote_category
0,Avatar,5PSNL1qE6VY,8611952.0,45703.0,2137.0,7631.0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,...,"['culture clash', 'future', 'space war', 'spac...","['English', 'Español']","['Ingenious Film Partners', 'Twentieth Century...","['United States of America', 'United Kingdom']",2009.0,12.0,2000.0,1,237.0,good
1,Pirates of the Caribbean: At World's End,HKSZtp_OGHY,2678687.0,16962.0,302.0,0.0,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,...,"['ocean', 'drug abuse', 'exotic island', 'east...",['English'],"['Walt Disney Pictures', 'Jerry Bruckheimer Fi...",['United States of America'],2007.0,5.0,2000.0,1,300.0,average
2,Spectre,ujmoYyEyDP8,5721876.0,14116.0,353.0,673.0,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,...,"['spy', 'based on novel', 'secret agent', 'seq...","['Français', 'English', 'Español', 'Italiano',...","['Columbia Pictures', 'Danjaq', 'B24']","['United Kingdom', 'United States of America']",2015.0,10.0,2010.0,1,245.0,average
3,The Dark Knight Rises,GokKUqLcvD8,38826999.0,118022.0,3683.0,49919.0,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,...,"['dc comics', 'crime fighter', 'terrorist', 's...",['English'],"['Legendary Pictures', 'Warner Bros.', 'DC Ent...",['United States of America'],2012.0,7.0,2010.0,1,250.0,good
4,John Carter,nlvYKl1fjBI,5065037.0,11752.0,748.0,652.0,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,...,"['based on novel', 'mars', 'medallion', 'space...",['English'],['Walt Disney Pictures'],['United States of America'],2012.0,3.0,2010.0,1,260.0,average


In [111]:
ratings_labels = movies['vote_category'].values

In [112]:
movies_knn = movies[['views', 'likes', 'dislikes', 'comments', 'budget',
       'runtime', 'vote_count', 'year', 'month', 'homepage_binary',
       'budget_mln']]

In [114]:
X = movies_knn.values
y = ratings_labels

In [115]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [116]:
from sklearn import metrics

In [117]:
def knn_cluster(k_values):
    for k in k_values:
        knn = KNeighborsClassifier(n_neighbors = k)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print(metrics.accuracy_score(y_test, y_pred))
        print(metrics.classification_report(y_test, y_pred))
        


k_values = [1, 3, 5, 7, 9]
knn_cluster(k_values)

0.663891779396462
              precision    recall  f1-score   support

     average       0.78      0.80      0.79       741
         bad       0.23      0.20      0.21        46
        good       0.21      0.20      0.20       170
       great       0.00      0.00      0.00         4

    accuracy                           0.66       961
   macro avg       0.31      0.30      0.30       961
weighted avg       0.65      0.66      0.66       961

0.7180020811654526
              precision    recall  f1-score   support

     average       0.77      0.91      0.83       741
         bad       0.18      0.04      0.07        46
        good       0.21      0.10      0.14       170
       great       0.00      0.00      0.00         4

    accuracy                           0.72       961
   macro avg       0.29      0.26      0.26       961
weighted avg       0.64      0.72      0.67       961

0.7471383975026015
              precision    recall  f1-score   support

     average       

/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classificat

              precision    recall  f1-score   support

     average       0.77      0.96      0.86       741
         bad       0.33      0.09      0.14        46
        good       0.18      0.03      0.05       170
       great       0.00      0.00      0.00         4

    accuracy                           0.75       961
   macro avg       0.32      0.27      0.26       961
weighted avg       0.64      0.75      0.68       961

0.7658688865764828
              precision    recall  f1-score   support

     average       0.78      0.98      0.87       741
         bad       0.50      0.09      0.15        46
        good       0.29      0.02      0.04       170
       great       0.00      0.00      0.00         4

    accuracy                           0.77       961
   macro avg       0.39      0.27      0.26       961
weighted avg       0.67      0.77      0.68       961



/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [118]:
# from mpl_toolkits.mplot3d import Axes3D

# fig = plt.figure(1, figsize=(20, 15))
# ax = Axes3D(fig, elev=48, azim=134)
# ax.scatter(X[:, 0], X[:, 1], X[:, 2], c = y,
#            cmap=plt.cm.Set1, edgecolor='k', s = X[:, 4]*50)

# for name, label in [('Virginica', 0), ('Setosa', 1), ('Versicolour', 2)]:
#     ax.text3D(X[y == label, 0].mean(),
#               X[y == label, 1].mean(),
#               X[y == label, 2].mean(), name,
#               horizontalalignment='center',
#               bbox=dict(alpha=.5, edgecolor='w', facecolor='w'),size=25)

# ax.set_title("3D visualization", fontsize=40)
# ax.set_xlabel("Sepal Length [cm]", fontsize=25)
# ax.w_xaxis.set_ticklabels([])
# ax.set_ylabel("Sepal Width [cm]", fontsize=25)
# ax.w_yaxis.set_ticklabels([])
# ax.set_zlabel("Petal Length [cm]", fontsize=25)
# ax.w_zaxis.set_ticklabels([])

# plt.show()

In [119]:
len(movies[movies["vote_category"] == "great"])

8